In [ ]:
import pandas as pd
import numpy as np
from os import path, makedirs

In [ ]:
base_path = path.join('data_output', 'urbanismo')
if not path.exists(base_path):
    makedirs(base_path)
def _normalize_headers(cols):
    return [str(c).strip() for c in cols]
def _clean_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = _normalize_headers(df.columns)
    for c in df.columns:
        if df[c].dtype == object:
            df[c] = df[c].astype(str).str.strip().replace({"": np.nan})
    return df

In [ ]:
df_atd = pd.read_csv("dados/urbanismo/atendimentos_viagens.csv",
                     sep=";", dtype=str)

df_loa = pd.read_csv("dados/urbanismo/loa_atende.csv",
                     sep=";", dtype=str)

In [ ]:
# Ajustar para formato iso data ex: AAAA-MM-DD
df_atd['data'] = pd.to_datetime(df_atd['data']).dt.strftime('%Y-%m-%d')
df_atd = _clean_df(df_atd)
df_atd

In [ ]:
df_loa

In [ ]:
base_path = path.join('data_output', 'urbanismo')

if not path.exists(base_path):
    makedirs(base_path)

for name, df in [
                 ('atendimentos_viagens', df_atd),
                 ('loa_atende', df_loa),
                 ]:
    filepath = path.join(base_path, f'{name}.csv')

    df.to_csv(filepath,
              index=False,
              sep=';',
              decimal=',',
              encoding='latin1')